# Dermatology Project

## Extract bottleneck features

In [1]:
import numpy as np
from sklearn.datasets import load_files       
from keras.utils import np_utils

def load_dataset(path):
    data = load_files(path)
    img_files = np.array(data['filenames'])
    img_targets = np_utils.to_categorical(np.array(data['target']), 3)
    return img_files, img_targets

Using TensorFlow backend.


In [2]:
train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test')

print("{} training images\n{} validation images\n{} test images".format(
    train_files.size, valid_files.size, test_files.size))

2000 training images
150 validation images
600 test images


In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(299, 299))
    # convert PIL.Image.Image type to 3D tensor with shape (299, 299, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 299, 299, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensors(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [9]:
from keras.applications.xception import Xception, preprocess_input

def extract_xception(file_paths):
    tensors = preprocess_input(paths_to_tensors(file_paths).astype('float32'))
    return Xception(weights='imagenet', include_top=False).predict(tensors, batch_size=32)


In [10]:
train_xception = extract_xception(train_files)
valid_xception = extract_xception(valid_files)
test_xception = extract_xception(test_files)

100%|██████████| 2000/2000 [03:29<00:00,  9.55it/s]


83689472/83683744 [==============================] - 59s 1us/step


100%|██████████| 600/600 [02:45<00:00,  3.63it/s]


In [15]:
np.savez_compressed('bottleneck_features/xception_features.npz',
                    train=train_xception, valid=valid_xception, test=test_xception)

print('Xception features shape:', train_xception.shape[1:])

Xception features shape: (10, 10, 2048)


## Build model

In [148]:
import numpy as np
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from keras.models import Sequential

In [121]:
bottleneck_features = np.load('bottleneck_features/xception_features.npz')
train_Xception = bottleneck_features['train']
valid_Xception = bottleneck_features['valid']
test_Xception = bottleneck_features['test']

In [202]:
Xception_model = Sequential()
Xception_model.add(GlobalAveragePooling2D(input_shape=train_Xception.shape[1:]))
# Xception_model.add(Dropout(0.2))
Xception_model.add(Dense(3, activation='softmax'))
Xception_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_45  (None, 2048)              0         
_________________________________________________________________
dense_74 (Dense)             (None, 3)                 6147      
Total params: 6,147
Trainable params: 6,147
Non-trainable params: 0
_________________________________________________________________


In [203]:
rmsprop = optimizers.RMSprop(lr=0.0005)
Xception_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [204]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.Xception.hdf5', 
                               verbose=1, save_best_only=True)

Xception_model.fit(train_Xception, train_targets, 
          validation_data=(valid_Xception, valid_targets),
          epochs=10, batch_size=10, callbacks=[checkpointer], verbose=1)

Train on 2000 samples, validate on 150 samples
Epoch 1/10
2000/2000 [==============================] - 3s 2ms/step - loss: 0.7507 - acc: 0.6950 - val_loss: 0.7755 - val_acc: 0.7000

Epoch 00001: val_loss improved from inf to 0.77546, saving model to saved_models/weights.best.Xception.hdf5
Epoch 2/10
2000/2000 [==============================] - 1s 527us/step - loss: 0.6390 - acc: 0.7425 - val_loss: 0.8602 - val_acc: 0.6600

Epoch 00002: val_loss did not improve
Epoch 3/10
2000/2000 [==============================] - 1s 538us/step - loss: 0.5943 - acc: 0.7630 - val_loss: 0.7730 - val_acc: 0.6867

Epoch 00003: val_loss improved from 0.77546 to 0.77296, saving model to saved_models/weights.best.Xception.hdf5
Epoch 4/10
2000/2000 [==============================] - 1s 542us/step - loss: 0.5667 - acc: 0.7675 - val_loss: 0.7700 - val_acc: 0.6600

Epoch 00004: val_loss improved from 0.77296 to 0.77000, saving model to saved_models/weights.best.Xception.hdf5
Epoch 5/10
2000/2000 [===============

## Test model

In [205]:
Xception_model.load_weights('saved_models/weights.best.Xception.hdf5')
Xception_predictions = [np.argmax(Xception_model.predict(np.expand_dims(feature, axis=0))) for feature in test_Xception]

# report test accuracy
test_accuracy = 100 * np.sum(
    np.array(Xception_predictions)==np.argmax(test_targets, axis=1)) / len(Xception_predictions)
print('Test accuracy: {:.4f}'.format(test_accuracy))

Test accuracy: 71.3333


In [206]:
Xception_predictions

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 2,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 0,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 0,
 1,
 1,
 0,
 1,
